In [30]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [31]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [32]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [33]:
tools=[wiki,arxiv]

In [34]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [35]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12f9ed300>, search_kwargs={})

In [36]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [37]:
tools = [wiki, arxiv]  # Remove retriever_tool - it's causing crashes
print(f"Using safe tools: {[tool.name for tool in tools]}")

Using safe tools: ['wikipedia', 'arxiv']


In [38]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/pedrodelafuente/Langchain Projects/1-QA_Chatbot/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [39]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import openai
import os

groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant")



In [40]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
prompt = hub.pull("hwchase17/react")
print("Prompt loaded")

Prompt loaded


In [41]:
## Agents
agent = create_react_agent(llm, tools, prompt)  # ✅ Correct function
print("Agent created successfully")
agent

Agent created successfully


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.', 'tool_names': 'wikipedia, arxiv'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the followin

In [42]:
from langchain.agents import AgentExecutor

# Add safety limits to prevent crashes
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3,           # Limit reasoning steps
    max_execution_time=30,      # 60 second timeout
    early_stopping_method="force",  # Stop early if needed
    handle_parsing_errors=True,   # Handle LLM parsing errors gracefully
    return_intermediate_steps=True
)

print("Agent executor created successfully!")
agent_executor

Agent executor created successfully!


AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.', 'tool_names': 'wikipedia, arxiv'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following q

In [43]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Thought: I'm not familiar with Langsmith. I should try to find more information about it.
Action: wikipedia
Action Input: LangsmithPage: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languagQuestion: Tell me about Langsmith
Thought: I'm not familiar with Langsmith. I should try to find more information about it.
Action: wikipedia
Action Input: LangsmithPage: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languagQuestion: Tell me about Langsmith
Thought: I'm not familiar with Langsmith. I should try to find more information about it.
Action: wikipedia
Action Input: Langsmit

{'input': 'Tell me about Langsmith',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='Langsmith', log="Thought: I'm not familiar with Langsmith. I should try to find more information about it.\nAction: wikipedia\nAction Input: Langsmith"),
   "Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag"),
  (AgentAction(tool='wikipedia', tool_input='Langsmith', log="Question: Tell me about Langsmith\nThought: I'm not familiar with Langsmith. I should try to find more information about it.\nAction: wikipedia\nAction Input: Langsmith"),
   "Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integratio

In [44]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to find a reliable source that explains what machine learning is.
Action: wikipedia
Action Input: machine learningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witQuestion: What is machine learning
Thought:Thought: To answer the question, I need to find a reliable source that explains what machine learning is.
Action: wikipedia
Action Input: machine learningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witQuestion: What is machine learning
Thought: To answer the question, I need to find a reliable source that exp

{'input': 'What is machine learning',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='machine learning', log='Thought: To answer the question, I need to find a reliable source that explains what machine learning is.\nAction: wikipedia\nAction Input: machine learning'),
   'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit'),
  (AgentAction(tool='wikipedia', tool_input='machine learning', log='Question: What is machine learning\nThought:Thought: To answer the question, I need to find a reliable source that explains what machine learning is.\nAction: wikipedia\nAction Input: machine learning'),
   'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concern

In [45]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...
Thought: I need to find a paper with the ID 1706.03762 on arXiv.org.
Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cQuestion: What's the paper 1706.03762 about?
Thought: I need to find a paper with the ID 1706.03762 on arXiv.org.
Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cQuestion: What's the paper 1706.03762 about?
Thought: I need to find a paper with the ID 1706.03762 on arXiv.org.
Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You

{'input': "What's the paper 1706.03762 about?",
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='arxiv', tool_input='1706.03762', log='Thought: I need to find a paper with the ID 1706.03762 on arXiv.org.\nAction: arxiv\nAction Input: 1706.03762'),
   'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant sequence transduction models are based on c'),
  (AgentAction(tool='arxiv', tool_input='1706.03762', log="Question: What's the paper 1706.03762 about?\nThought: I need to find a paper with the ID 1706.03762 on arXiv.org.\nAction: arxiv\nAction Input: 1706.03762"),
   'Published: 2023-08-02\nTitle: Attention Is All You Need\nAuthors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin\nSummary: The dominant